In [4]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import logging
import requests
import os

from samgeo.fast_sam import SamGeo  # Import the SamGeo class for raster-to-vector conversion
import matplotlib.pyplot as plt

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

app = Flask(__name__)

# Configure CORS more specifically
CORS(app, resources={
    r"/test": {
        "origins": ["http://localhost:3000"],  # Adjust this to match your frontend URL
        "methods": ["POST"],
        "allow_headers": ["Content-Type"]
    }
})

@app.route('/test', methods=['POST'])
def test():
    try:
        
        data = request.get_json()
        if data is None:
            return jsonify({"status": "error", "message": "No JSON data received"}), 400
        
        signed_url = data.get('signedUrl')
            
        fileName = data.get('fileName')

        textPrompt = data.get('textPrompt')

        listOfPoints= data.get('listOfPoints')

        # Download the file from the signed URL
        response = requests.get(signed_url, stream=True)
        
    
        output_folder = "./output"

        original_file_path = os.path.join(output_folder, fileName)

        # Download the file from the public URL
        response = requests.get(signed_url)

        # Save the file to the output folder
        with open(original_file_path, "wb") as f:
            f.write(response.content)

        output_raster = './output/output_raster.tif'  # Output path for the raster mask (GeoTIFF)
        output_vector = './output/output_vector.geojson'  # Output path for the vector file (GeoJSON)


        sam = SamGeo(model="FastSAM-x.pt")  # LangSAM for text-based segmentation


        
        # point prompt
        # image_path = './images/cat.jpg'
        sam.set_image(original_file_path)
        positive_points = [[450, 300], [400, 200], [500, 200], [450, 450]] 
        positive_points = listOfPoints
        negative_points = [[100, 100], [550, 600], [300, 350]]  # Areas to exclude (background and red object)
        # Define the labels for the points: 1 for positive points, 0 for negative points
        point_labels = [1, 1, 1, 1, 0, 0, 0]  # 1: Positive, 0: Negative
        sam.point_prompt(points=positive_points + negative_points, pointlabel=point_labels)
        # Save the result as an image (e.g., PNG)
        output_image = "./output/output_segmentation_with_points.png"
        sam.show_anns(output_image)


        # # Perform segmentation based on the text prompt using langsam
        # sam.predict(output_file_path, textPrompt, box_threshold=0.24, text_threshold=0.24)

        # # Save the result to a raster file (GeoTIFF)
        # sam.show_anns(
        #     cmap="Greys_r", 
        #     add_boxes=False, 
        #     alpha=1, 
        #     title="Segmentation Result", 
        #     blend=False, 
        #     output=output_raster
        # )

        # Convert the raster result to vector format
        sam.raster_to_vector(output_raster, output_vector)


        # sam = SamGeo(model="FastSAM-x.pt")  

        # sam.set_image(output_file_path)


        # sam.everything_prompt(output=output_raster) 


        # sam.raster_to_vector(output_raster, output_vector) 
        
        return jsonify({
            "status": "success",
            "received_signed_url": signed_url
        })
        
    except Exception as e:
        logger.exception("Error processing request")
        return jsonify({
            "status": "error", 
            "message": str(e),
            "type": str(type(e).__name__)
        }), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.10:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
Traceback (most recent call last):
  File "/opt/miniconda3/envs/FastSAM/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/miniconda3/envs/FastSAM/lib/python3.9/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/miniconda3/envs/FastSAM/lib/python3.9/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/opt/miniconda3/envs/FastSAM/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/opt/miniconda3/envs/FastSAM/lib/python3.9/site-

SystemExit: 1

/opt/miniconda3/envs/FastSAM/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
! pip install git+https://github.com/openai/CLIP.git
import clip
clip.available_models()

  Cloning https://github.com/openai/CLIP.git to /private/var/folders/xc/f09q_z6943z0mvqw0lrn19dw0000gp/T/pip-req-build-e4kxf9tr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/xc/f09q_z6943z0mvqw0lrn19dw0000gp/T/pip-req-build-e4kxf9tr
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [14]:
! pip install git+https://github.com/openai/CLIP.git
clip.available_models()

from samgeo.fast_sam import SamGeo  # Import the SamGeo class for raster-to-vector conversion
from samgeo import tms_to_geotiff
import matplotlib.pyplot as plt
import requests
import os

# Specify the model path and image path
model_path = './weights/FastSAM-x.pt'  # Path to the FastSAM model
output_raster = './output/output_raster.jpg'  # Output path for the raster mask (GeoTIFF)
output_vector = './output/output_vector.geojson'  # Output path for the vector file (GeoJSON)


sam = SamGeo(model="FastSAM-x.pt")  # Initialize SamGeo with the FastSAM model


# Define the local output path
output_folder = "./output"
output_file_path = os.path.join(output_folder, "initial_file.tif")


image_path = './images/cat.jpg'
sam.set_image(image_path)



# #"everything" prompt
# sam.everything_prompt(output=output_raster)  # Output the mask as a GeoTIFF

# text prompt
sam.text_prompt(text = "just the eyes", output = output_raster)
sam.show_anns(output_raster.png)

# # point prompt
# positive_points = [[450, 300], [400, 200], [500, 200], [450, 450]]  # Areas to include (head, ears, body)
# negative_points = [[100, 100], [550, 600], [300, 350]]  # Areas to exclude (background and red object)

# # Define the labels for the points: 1 for positive points, 0 for negative points
# point_labels = [1, 1, 1, 1, 0, 0, 0]  # 1: Positive, 0: Negative
# sam.point_prompt(points=positive_points + negative_points, pointlabel=point_labels)
# # Save the result as an image (e.g., PNG)
# output_image = "cat_segmentation_with_points.png"
# sam.show_anns(output_image)

# Step 3: Convert the raster segmentation result to a vector format
# sam.raster_to_vector(output_raster, output_vector)  # Convert GeoTIFF to GeoJSON




  Cloning https://github.com/openai/CLIP.git to /private/var/folders/xc/f09q_z6943z0mvqw0lrn19dw0000gp/T/pip-req-build-3r6wrj0t
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/xc/f09q_z6943z0mvqw0lrn19dw0000gp/T/pip-req-build-3r6wrj0t
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done



image 1/1 /Users/xaviertan/Documents/Nika/Code/FastSAM/images/cat.jpg: 1024x576 38 objects, 1284.8ms
Speed: 3.3ms preprocess, 1284.8ms inference, 17.5ms postprocess per image at shape (1, 3, 1024, 1024)
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 41 32768


NameError: name 'clip' is not defined